In [2]:
data = [1,2,3,4,5]
def num_func(x):
    return x**2/2
print(list(map(num_func,data)))
print(type(list(map(num_func,data))))

[0.5, 2.0, 4.5, 8.0, 12.5]
<class 'list'>


In [3]:
from functools import reduce
def num_add(a,b):
    return a+b
data = [1,2,3,4,5]
print(reduce(num_add,data))
print(type(reduce(num_add,data)))

15
<class 'int'>


In [4]:
def more_than_15(x):
    return x > 15
data = [1,10,100]
print(list(filter(more_than_15,data)))
print(type(list(filter(more_than_15,data))))

[100]
<class 'list'>
